Create participation map using the IP from the lists of abstract.

We installed the following packages:
* [pycountry](https://pypi.python.org/pypi/pycountry), to work on the name country names;
* [python-geoip](http://pythonhosted.org/python-geoip/), to access to GeoIP databases.

Matplotlib is used to create the figure and shapefile to read the country shapefile.

In [1]:
import os
import numpy as np
import matplotlib as mpl
import re
from collections import Counter
import liegecolloquium
import pycountry
import logging
import PIL
from mpl_toolkits.basemap import Basemap

In [2]:
logger = logging.getLogger('participation-map')
logger.setLevel(logging.DEBUG)

# Input files

In [3]:
countryshapefile = "/home/ctroupin/Data/ShapeFiles/TM_WORLD_BORDERS_SIMPL-0.3/TM_WORLD_BORDERS_SIMPL-03.shp"
abstractfile = '../data/abstracts/abstractslist2013.txt'
figdir = "../figures/abstractmaps/"
figname = os.path.join(figdir, "".join((os.path.basename(abstractfile).split('.')[0],'_map')))

Check if files and directories exist:

In [4]:
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Create a list of countries from the list of abstracts

We use a regular expression to extract the IP and the date, then we call `geolite2` to get the country from the IP.    
We build a list of country iso-names (available using the `geolite2` response).

In [5]:
countrylist = liegecolloquium.countries_from_abstract_list(abstractfile)

Number of abstracts in the list: 175
INFO:locator:Number of abstracts in the list: 175
Number of identified countries: 175
INFO:locator:Number of identified countries: 175


# Make the plot

Create the projection

In [6]:
m = Basemap(projection='mill',llcrnrlat=-80,urcrnrlat=85,\
            llcrnrlon=-180,urcrnrlon=180,resolution='c')

In [7]:
mpl.rcParams.update({'font.size': 22})
liegecolloquium.make_country_map(countrylist, countryshapefile, m,
                                 "./tesssss.png", year='2013', nparticipants=12)

In [9]:
pp= pycountry.countries.lookup('de')

In [10]:
pp.alpha_2

'DE'